The following query was used to generate a CSV file sourced here that will export again to a different format for the purposes of creating a pseudo-timeline in Excel.

This query was run against version 14 of the database.

```
SELECT files.id as files_id,
       files.year,
	   files.month,
       facilities.id AS facility_id,
       facilities.facility_parent_id,
       facilities.facilityname,
	   facilities.country,
	   facilities.governorate,
	   facilities.district,
	   facilities.subdistrict,
	   facilities.facility_type,
	   COUNT(full_raw_scrubbed.id) AS num_records
FROM files
JOIN facilities on files.facility_id = facilities.id
JOIN full_raw_scrubbed on files.id = full_raw_scrubbed.a_file_id

WHERE files.facility_id IS NOT NULL 
AND files.month IS NOT NULL
AND files.skipped = 0
AND files.ignore = 0

ORDER BY files.year ASC, files.month ASC, facilities.facilityname ASC;
```

In [1]:
import csv

In [2]:
source = "02-DEC-2017_facility_timeline_data.csv"

In [11]:
data = []

intcols = [
    "facility_id",
    "facility_parent_id",
    "month",
    "year",
    "num_records",
]

with open(source,'r', encoding='utf-8-sig') as csvfile:
    csvreader = csv.reader(csvfile)
    header = next(csvreader)
    for row in csvreader:
        rowdict = dict(zip(header,row))
        for col in intcols:
            try:
                rowdict[col] = int(rowdict[col])
            except:
                pass
        data.append(rowdict)

In [26]:
data[0]

{'country': 'Syria',
 'district': '',
 'facility_id': 66,
 'facility_parent_id': '',
 'facility_type': 'RH',
 'facilityname': 'Rawan Birth Center',
 'governorate': 'Daraa',
 'month': 7,
 'num_records': 483,
 'subdistrict': '',
 'year': 2014}

In [25]:
poss_dates = []

for x in range(2014,2018):
    for y in range(1,13):
        pd = str(x) + "-" + str(y)
        poss_dates.append(pd)

['2014-1', '2014-2', '2014-3', '2014-4', '2014-5', '2014-6', '2014-7', '2014-8', '2014-9', '2014-10', '2014-11', '2014-12', '2015-1', '2015-2', '2015-3', '2015-4', '2015-5', '2015-6', '2015-7', '2015-8', '2015-9', '2015-10', '2015-11', '2015-12', '2016-1', '2016-2', '2016-3', '2016-4', '2016-5', '2016-6', '2016-7', '2016-8', '2016-9', '2016-10', '2016-11', '2016-12', '2017-1', '2017-2', '2017-3', '2017-4', '2017-5', '2017-6', '2017-7', '2017-8', '2017-9', '2017-10', '2017-11', '2017-12']


In [48]:
lookup = {}

transfer_keys = [
    "country",
    "governorate",
    "district",
    "subdistrict",
    "facility_id",
    "facility_parent_id",
    "facility_type"
]

for rec in data:
    fn = rec["facilityname"]
    
    # Setup structure
    if fn not in lookup.keys():
        lookup[fn] = {"facilityname":fn}
        for k in transfer_keys:
            lookup[fn][k] = rec[k]
            
    count_key = str(rec['year']) + "-" + str(rec['month'])
    
    lookup[fn][count_key] = rec["num_records"]
    if lookup[fn][count_key] == 0:
        lookup[fn][count_key] = ''

In [49]:
for fac in lookup.keys():
    for pd in poss_dates:
        if pd in lookup[fac].keys():
            continue
        else:
            lookup[fac][pd] = ''

In [50]:
lookup['Rawan Birth Center']

{'2014-1': '',
 '2014-10': 357,
 '2014-11': 358,
 '2014-12': 329,
 '2014-2': '',
 '2014-3': '',
 '2014-4': '',
 '2014-5': '',
 '2014-6': '',
 '2014-7': 483,
 '2014-8': 407,
 '2014-9': 373,
 '2015-1': 152,
 '2015-10': '',
 '2015-11': '',
 '2015-12': '',
 '2015-2': 75,
 '2015-3': 851,
 '2015-4': 276,
 '2015-5': 414,
 '2015-6': '',
 '2015-7': '',
 '2015-8': '',
 '2015-9': '',
 '2016-1': '',
 '2016-10': '',
 '2016-11': '',
 '2016-12': '',
 '2016-2': '',
 '2016-3': '',
 '2016-4': '',
 '2016-5': '',
 '2016-6': '',
 '2016-7': '',
 '2016-8': '',
 '2016-9': '',
 '2017-1': '',
 '2017-10': '',
 '2017-11': '',
 '2017-12': '',
 '2017-2': '',
 '2017-3': '',
 '2017-4': '',
 '2017-5': '',
 '2017-6': '',
 '2017-7': '',
 '2017-8': '',
 '2017-9': '',
 'country': 'Syria',
 'district': '',
 'facility_id': 66,
 'facility_parent_id': '',
 'facility_type': 'RH',
 'facilityname': 'Rawan Birth Center',
 'governorate': 'Daraa',
 'subdistrict': ''}

In [51]:
header = [
    "country",
    "governorate",
    "district",
    "subdistrict",
    "facility_type",
    "facility_id",
    "facility_parent_id",
    "facilityname"
]

header = header + poss_dates

In [52]:
with open("02-DEC-2017_Output_SAMS_raw_facilities_records_by_date.csv",'w') as outfile:
    outfile.write(",".join(header))
    outfile.write("\n")
    for fac in lookup.keys():
        row = [str(lookup[fac][k]) for k in header]
        outfile.write(",".join(row))
        outfile.write("\n")